In [1]:
import gc
import pandas as pd
import numpy as np
from sklearn import metrics, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
import sys
import warnings
import itertools

if not sys.warnoptions:
    warnings.simplefilter("ignore")

path = "/home/darragh/avito/data/"
#path = '/Users/dhanley2/Documents/avito/data/'
#path = '/home/ubuntu/avito/data/'
#data_path = 'data/'

In [8]:
#cnames = ['item_id', 'deal_probability']
#rnn27 = pd.concat([pd.read_csv(path+'../sub/rnndhCV_1206B_trn.csv', header = None, names = cnames), 
#                   pd.read_csv(path+'../sub/rnndhCV_1206A_tst.csv', header = None, names = cnames)])
#rnn27.to_csv(path+'../sub/rnnCV_1206.csv.gz', compression='gzip', index = False)

In [10]:
#lgb25 = pd.read_csv('../lgCV_2505.csv.gz', compression='gzip')
lgb02A = pd.read_csv(path+'../sub/lgCV_0206A.csv.gz', compression='gzip')
lgb09 = pd.read_csv(path+'../sub/lgCV_0906.csv.gz', compression='gzip')
lgb10 = pd.read_csv(path+'../sub/lgCV_1006.csv.gz', compression='gzip')
lgb11A= pd.read_csv(path+'../sub/lgCV_1106A.csv.gz', compression='gzip')
lgb11D= pd.read_csv(path+'../sub/lgCV_1106D.csv.gz', compression='gzip')
lgb14= pd.read_csv(path+'../sub/lgCV_1406.csv.gz', compression='gzip')
lgb14A= pd.read_csv(path+'../sub/lgCV_1406A.csv.gz', compression='gzip')
lgb27 = pd.read_csv(path+'../sub/lgCV_2705B.csv.gz', compression='gzip')
lgb31 = pd.read_csv(path+'../sub/lgCV_3105.csv.gz', compression='gzip')
lgb02 = pd.read_csv(path+'../sub/lgCV_0206.csv.gz', compression='gzip')
rnn =   pd.read_csv(path+'../sub/rnnCV_2805.csv.gz', compression='gzip')
rnn27 = pd.read_csv(path+'../sub/rnnCV_2705A.csv.gz', compression='gzip')
rnn12 = pd.read_csv(path+'../sub/rnnCV_1206.csv.gz', compression='gzip')
mlp =   pd.read_csv(path+'../sub/mlpCV_2505.csv.gz', compression='gzip')
truth = pd.read_csv(path+'train.csv.zip', compression='zip', parse_dates = ["activation_date"])
y =     truth['deal_probability'].values
truth.drop('deal_probability', 1)
test =  pd.read_csv(path+'test.csv.zip', compression='zip', parse_dates = ["activation_date"])
test['deal_probability']=float('NAN') 
truth = pd.concat([truth,test[truth.columns]],axis=0)

In [11]:
lgb14['deal_probability'] =  ( lgb14['deal_probability'].values + lgb14A['deal_probability'].values)*0.5

In [12]:
#lgb25.rename(columns={'deal_probability': 'lgb25_preds' }, inplace=True)
lgb31.rename(columns={'deal_probability': 'lgb31_preds' }, inplace=True)
lgb27.rename(columns={'deal_probability': 'lgb27_preds' }, inplace=True)
lgb02.rename(columns={'deal_probability': 'lgb02_preds' }, inplace=True)
lgb09.rename(columns={'deal_probability': 'lgb09_preds' }, inplace=True)
lgb10.rename(columns={'deal_probability': 'lgb10_preds' }, inplace=True)
lgb11D.rename(columns={'deal_probability': 'lgb11D_preds' }, inplace=True)
lgb11A.rename(columns={'deal_probability': 'lgb11A_preds' }, inplace=True)
lgb14.rename(columns={'deal_probability': 'lgb14_preds' }, inplace=True)
lgb02A.rename(columns={'deal_probability': 'lgb02A_preds' }, inplace=True)
rnn27.rename(columns={'deal_probability': 'rnn27_preds' }, inplace=True)
rnn12.rename(columns={'deal_probability': 'rnn12_preds' }, inplace=True)
mlp.rename(columns={'deal_probability': 'mlp_preds' }, inplace=True)
preds_df = lgb27.merge(rnn, on='item_id')\
                .merge(mlp, on='item_id')\
                .merge(lgb31, on='item_id')\
                .merge(lgb02, on='item_id')\
                .merge(lgb09, on='item_id')\
                .merge(lgb10, on='item_id')\
                .merge(lgb11A, on='item_id')\
                .merge(lgb11D, on='item_id')\
                .merge(lgb14, on='item_id')\
                .merge(lgb02A, on='item_id')\
                .merge(rnn27, on='item_id')\
                .merge(rnn12, on='item_id')\
                .merge(truth, on='item_id',how='left')

In [13]:
pred_cols = [col for col in preds_df.columns if ('_preds' in col) \
             and ('lgb27' not in col) and ('lgb02_' not in col) and ('lgb11A_' not in col)]

preds_df['preds_sum'] = preds_df[pred_cols].sum(axis=1)
pred_cols

['rnn_preds',
 'mlp_preds',
 'lgb31_preds',
 'lgb09_preds',
 'lgb10_preds',
 'lgb11D_preds',
 'lgb14_preds',
 'lgb02A_preds',
 'rnn27_preds',
 'rnn12_preds']

In [14]:
preds_df['price'].fillna(-1,inplace=True)
preds_df['max'] = np.max(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['min'] = np.min(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['avg'] = np.mean(np.array([preds_df[col] for col in pred_cols]),axis=0)
preds_df['std'] = np.std(np.array([preds_df[col] for col in pred_cols]),axis=0)

In [15]:
import itertools
for p1, p2 in itertools.combinations(pred_cols, 2):
    print('difference_%s__%s'%(p1,p2))
    preds_df['difference_%s__%s'%(p1,p2)] = preds_df[p2] - preds_df[p1]

difference_rnn_preds__mlp_preds
difference_rnn_preds__lgb31_preds
difference_rnn_preds__lgb09_preds
difference_rnn_preds__lgb10_preds
difference_rnn_preds__lgb11D_preds
difference_rnn_preds__lgb14_preds
difference_rnn_preds__lgb02A_preds
difference_rnn_preds__rnn27_preds
difference_rnn_preds__rnn12_preds
difference_mlp_preds__lgb31_preds
difference_mlp_preds__lgb09_preds
difference_mlp_preds__lgb10_preds
difference_mlp_preds__lgb11D_preds
difference_mlp_preds__lgb14_preds
difference_mlp_preds__lgb02A_preds
difference_mlp_preds__rnn27_preds
difference_mlp_preds__rnn12_preds
difference_lgb31_preds__lgb09_preds
difference_lgb31_preds__lgb10_preds
difference_lgb31_preds__lgb11D_preds
difference_lgb31_preds__lgb14_preds
difference_lgb31_preds__lgb02A_preds
difference_lgb31_preds__rnn27_preds
difference_lgb31_preds__rnn12_preds
difference_lgb09_preds__lgb10_preds
difference_lgb09_preds__lgb11D_preds
difference_lgb09_preds__lgb14_preds
difference_lgb09_preds__lgb02A_preds
difference_lgb09_pre

In [16]:
gc.collect()

161

In [17]:
preds_df.head()

,item_id,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb09_preds,lgb10_preds,lgb11A_preds,lgb11D_preds,...,difference_lgb11D_preds__lgb14_preds,difference_lgb11D_preds__lgb02A_preds,difference_lgb11D_preds__rnn27_preds,difference_lgb11D_preds__rnn12_preds,difference_lgb14_preds__lgb02A_preds,difference_lgb14_preds__rnn27_preds,difference_lgb14_preds__rnn12_preds,difference_lgb02A_preds__rnn27_preds,difference_lgb02A_preds__rnn12_preds,difference_rnn27_preds__rnn12_preds
0,b912c3c6a6ad,0.091656,0.069891,0.091223,0.083219,0.090855,0.109624,0.107764,0.095101,0.092921,...,-0.008715,0.016697,-0.024590,-0.016043,0.025412,-0.015875,-0.007329,-0.041287,-0.032740,0.008546
1,2dac0150717d,0.147801,0.075968,0.211844,0.157973,0.119609,0.157466,0.161107,0.140269,0.144395,...,-0.008224,0.015649,-0.073757,-0.055836,0.023873,-0.065534,-0.047612,-0.089406,-0.071485,0.017921
2,ba83aefab5dc,0.186261,0.167167,0.236579,0.227048,0.258980,0.245236,0.219107,0.234076,0.222678,...,-0.023230,0.018658,-0.060754,-0.074653,0.041888,-0.037524,-0.051423,-0.079412,-0.093311,-0.013899
3,02996f1dd2ea,0.241699,0.260876,0.383699,0.231447,0.284130,0.289101,0.255519,0.241802,0.271355,...,-0.012286,0.065100,0.007448,-0.023032,0.077387,0.019735,-0.010746,-0.057652,-0.088132,-0.030481
4,7c90be56d2ab,0.407423,0.431520,0.420657,0.412563,0.430581,0.511145,0.511676,0.504849,0.503961,...,-0.002923,-0.081972,-0.153677,-0.082617,-0.079049,-0.150753,-0.079694,-0.071704,-0.000645,0.071059


### Train scores

In [18]:
pred_cols = [col for col in preds_df.columns if ('preds' in col) and ('difference' not in col)]
pred_cols

['lgb27_preds',
 'rnn_preds',
 'mlp_preds',
 'lgb31_preds',
 'lgb02_preds',
 'lgb09_preds',
 'lgb10_preds',
 'lgb11A_preds',
 'lgb11D_preds',
 'lgb14_preds',
 'lgb02A_preds',
 'rnn27_preds',
 'rnn12_preds',
 'preds_sum']

In [19]:
idx = preds_df['deal_probability']==preds_df['deal_probability']
print(idx.value_counts())
for col in [c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]:
    print('RMSE %s: '%(col), np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, preds_df[col][idx].values)))

True     1503424
False     508438
Name: deal_probability, dtype: int64
RMSE lgb27_preds:  0.21681527582821059
RMSE rnn_preds:  0.21771476573780568
RMSE mlp_preds:  0.2187544351068867
RMSE lgb31_preds:  0.21681135578306754
RMSE lgb02_preds:  0.21625386267187377
RMSE lgb09_preds:  0.21581543265040168
RMSE lgb10_preds:  0.21533498019117664
RMSE lgb11A_preds:  0.21403812331071156
RMSE lgb11D_preds:  0.21392147604708842
RMSE lgb14_preds:  0.21329729304260686
RMSE lgb02A_preds:  0.21627136604335995
RMSE rnn27_preds:  0.21698040871120608
RMSE rnn12_preds:  0.2167114508342396


In [20]:
foldls = [["2017-03-15", "2017-03-16", "2017-03-17"], \
       ["2017-03-18", "2017-03-19", "2017-03-20"], \
       ["2017-03-21", "2017-03-22", "2017-03-23"], \
       ["2017-03-24", "2017-03-25", "2017-03-26"], \
        ["2017-03-27", "2017-03-28", "2017-03-29", \
            "2017-03-30", "2017-03-31", "2017-04-01", \
            "2017-04-02", "2017-04-03","2017-04-07"]]
foldls = [[pd.to_datetime(d) for d in f] for f in foldls]
preds_df['fold'] = -1
for t, fold in enumerate(foldls):
    preds_df['fold'][preds_df.activation_date.isin(fold)] = t

In [21]:
scores = []
for col in [c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]:
    lstmp = [col]
    for i in range(5):
        idx = preds_df['fold']==i
        lstmp.append(np.sqrt(metrics.mean_squared_error(preds_df['deal_probability'][idx].values, \
                                                        preds_df[col][idx].values)))
    scores.append(lstmp)
pd.DataFrame(scores, columns = ['Model']+['Fold%s'%(i) for i in range(5)])

,Model,Fold0,Fold1,Fold2,Fold3,Fold4
0,lgb27_preds,0.216848,0.217096,0.217923,0.217597,0.213692
1,rnn_preds,0.218756,0.218966,0.215764,0.219554,0.214747
2,mlp_preds,0.219660,0.220088,0.216672,0.220347,0.216385
3,lgb31_preds,0.216537,0.216643,0.219230,0.217169,0.213430
4,lgb02_preds,0.216371,0.216447,0.217354,0.217032,0.213158
5,lgb09_preds,0.215514,0.215678,0.218418,0.216073,0.212293
6,lgb10_preds,0.215240,0.215533,0.217072,0.215802,0.212023
7,lgb11A_preds,0.215011,0.215205,0.211879,0.215771,0.211735
8,lgb11D_preds,0.214818,0.215074,0.211837,0.215720,0.211549
9,lgb14_preds,0.214204,0.214441,0.211074,0.215146,0.211051


### Correlations in test and train

In [22]:
# Test correlation
preds_df[~preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]].corr()

,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb09_preds,lgb10_preds,lgb11A_preds,lgb11D_preds,lgb14_preds,lgb02A_preds,rnn27_preds,rnn12_preds
lgb27_preds,1.000000,0.915952,0.919171,0.990750,0.988682,0.978331,0.978298,0.966819,0.965873,0.961540,0.983143,0.916804,0.920576
rnn_preds,0.915952,1.000000,0.929710,0.910602,0.913307,0.908867,0.913286,0.915655,0.913936,0.912870,0.911443,0.985360,0.978274
mlp_preds,0.919171,0.929710,1.000000,0.914307,0.916343,0.909521,0.913883,0.915367,0.913018,0.911656,0.913600,0.925010,0.929518
lgb31_preds,0.990750,0.910602,0.914307,1.000000,0.991984,0.981891,0.981375,0.966656,0.965897,0.961575,0.986851,0.914204,0.916939
lgb02_preds,0.988682,0.913307,0.916343,0.991984,1.000000,0.982596,0.982052,0.970858,0.969933,0.966325,0.987434,0.916726,0.920105
lgb09_preds,0.978331,0.908867,0.909521,0.981891,0.982596,1.000000,0.990447,0.976755,0.976540,0.972392,0.988708,0.912375,0.915090
lgb10_preds,0.978298,0.913286,0.913883,0.981375,0.982052,0.990447,1.000000,0.982841,0.982033,0.977396,0.985858,0.916436,0.920034
lgb11A_preds,0.966819,0.915655,0.915367,0.966656,0.970858,0.976755,0.982841,1.000000,0.992400,0.988703,0.971893,0.918727,0.923137
lgb11D_preds,0.965873,0.913936,0.913018,0.965897,0.969933,0.976540,0.982033,0.992400,1.000000,0.989978,0.971846,0.917196,0.921322
lgb14_preds,0.961540,0.912870,0.911656,0.961575,0.966325,0.972392,0.977396,0.988703,0.989978,1.000000,0.967560,0.916160,0.920547


In [23]:
# Train correlation
preds_df[preds_df['deal_probability'].isnull()][[c for c in preds_df.columns if ('_preds' in c)  and ('difference' not in c) ]].corr()

,lgb27_preds,rnn_preds,mlp_preds,lgb31_preds,lgb02_preds,lgb09_preds,lgb10_preds,lgb11A_preds,lgb11D_preds,lgb14_preds,lgb02A_preds,rnn27_preds,rnn12_preds
lgb27_preds,1.000000,0.930288,0.924283,0.991439,0.989502,0.979729,0.979341,0.975840,0.973725,0.970062,0.984742,0.930692,0.932951
rnn_preds,0.930288,1.000000,0.939951,0.927835,0.927867,0.926158,0.928367,0.927119,0.925632,0.924680,0.928496,0.988198,0.984113
mlp_preds,0.924283,0.939951,1.000000,0.922413,0.921927,0.918064,0.920496,0.919275,0.917496,0.916323,0.922121,0.935069,0.936609
lgb31_preds,0.991439,0.927835,0.922413,1.000000,0.993168,0.982874,0.982519,0.978877,0.976777,0.973170,0.988043,0.931468,0.932981
lgb02_preds,0.989502,0.927867,0.921927,0.993168,1.000000,0.984427,0.983525,0.980105,0.978173,0.975031,0.989577,0.931447,0.933333
lgb09_preds,0.979729,0.926158,0.918064,0.982874,0.984427,1.000000,0.991413,0.988267,0.986666,0.983504,0.989106,0.929748,0.931167
lgb10_preds,0.979341,0.928367,0.920496,0.982519,0.983525,0.991413,1.000000,0.992340,0.990473,0.986869,0.986654,0.931841,0.933902
lgb11A_preds,0.975840,0.927119,0.919275,0.978877,0.980105,0.988267,0.992340,1.000000,0.993474,0.990047,0.983556,0.930518,0.932743
lgb11D_preds,0.973725,0.925632,0.917496,0.976777,0.978173,0.986666,0.990473,0.993474,1.000000,0.991525,0.981933,0.929068,0.931133
lgb14_preds,0.970062,0.924680,0.916323,0.973170,0.975031,0.983504,0.986869,0.990047,0.991525,1.000000,0.978724,0.928224,0.930421


In [24]:
cols = [c for c in preds_df.columns if '_preds' in c]
cols += [c for c in preds_df.columns if 'difference' in c]
cols += ['price', 'max', 'min', 'avg', 'std']
categories = ['region', 'param_1', 'parent_category_name', 'category_name', 'param_2' ]
cols += categories
cols = list(set(cols))

In [25]:
for col in categories:
    preds_df[col] = LabelEncoder().fit_transform(preds_df[col].fillna("0"))
train_df = preds_df[~preds_df['deal_probability'].isnull()]
test_df = preds_df[preds_df['deal_probability'].isnull()]

In [26]:
n_estimators = 4000
train_X, valid_X, train_y, valid_y = train_test_split(train_df[cols], y, train_size=.8, random_state=12345)
eval_set = [(train_X,train_y),(valid_X,valid_y)]

In [27]:
len(train_X)

1202739

In [30]:
%%time
gc.collect()
clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.4,
                    num_leaves=32, 
                    learning_rate=.01)#, device='gpu')
clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)
# [1435]	training's rmse: 0.20979	valid_1's rmse: 0.212398
# [1502]	training's rmse: 0.209679	valid_1's rmse: 0.212317

Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.219409	valid_1's rmse: 0.219809
[200]	training's rmse: 0.212874	valid_1's rmse: 0.213563
[300]	training's rmse: 0.211679	valid_1's rmse: 0.212604
[400]	training's rmse: 0.211297	valid_1's rmse: 0.212425
[500]	training's rmse: 0.211066	valid_1's rmse: 0.212371
[600]	training's rmse: 0.210883	valid_1's rmse: 0.212352
[700]	training's rmse: 0.21072	valid_1's rmse: 0.212343
[800]	training's rmse: 0.210579	valid_1's rmse: 0.21234
[900]	training's rmse: 0.210443	valid_1's rmse: 0.212336
[1000]	training's rmse: 0.210307	valid_1's rmse: 0.212329
[1100]	training's rmse: 0.21018	valid_1's rmse: 0.212326
[1200]	training's rmse: 0.210049	valid_1's rmse: 0.212325
[1300]	training's rmse: 0.209924	valid_1's rmse: 0.212323
[1400]	training's rmse: 0.209802	valid_1's rmse: 0.21232
[1500]	training's rmse: 0.209681	valid_1's rmse: 0.212317
Early stopping, best iteration is:
[1502]	training's rmse: 0.209679	valid_1's rms

In [31]:
sorted(zip(clf.feature_importances_, train_X.columns ),key=lambda x: -x[0])

[(6212, 'param_1'),
 (3399, 'param_2'),
 (2797, 'region'),
 (1852, 'category_name'),
 (1463, 'lgb14_preds'),
 (1048, 'difference_rnn_preds__rnn27_preds'),
 (1015, 'avg'),
 (935, 'rnn27_preds'),
 (921, 'mlp_preds'),
 (841, 'rnn12_preds'),
 (815, 'difference_rnn_preds__rnn12_preds'),
 (769, 'difference_lgb31_preds__lgb14_preds'),
 (763, 'min'),
 (745, 'difference_lgb11D_preds__lgb14_preds'),
 (742, 'difference_lgb10_preds__lgb14_preds'),
 (705, 'difference_lgb14_preds__lgb02A_preds'),
 (627, 'lgb11D_preds'),
 (591, 'difference_mlp_preds__lgb31_preds'),
 (569, 'difference_lgb09_preds__lgb02A_preds'),
 (557, 'difference_rnn_preds__lgb14_preds'),
 (548, 'difference_rnn27_preds__rnn12_preds'),
 (530, 'difference_rnn_preds__mlp_preds'),
 (528, 'difference_lgb31_preds__lgb09_preds'),
 (517, 'price'),
 (512, 'lgb11A_preds'),
 (510, 'difference_lgb31_preds__lgb02A_preds'),
 (496, 'difference_mlp_preds__lgb02A_preds'),
 (484, 'difference_lgb10_preds__lgb02A_preds'),
 (483, 'difference_mlp_preds__

In [32]:
n_estimators = 1502
train_X = train_df[cols]
train_y = y
eval_set = [(train_X,train_y)]
len(train_X)

1503424

In [33]:
%%time
y_predls = []
for i in range(3):
    clf = LGBMRegressor(n_estimators=n_estimators, 
                    max_depth=-1, 
                    feature_fraction= 0.4,
                    num_leaves=32, 
                    seed = i, 
                    learning_rate=.01)#, device='gpu')
    clf.fit(train_X, train_y, early_stopping_rounds=80, 
        eval_set=eval_set, eval_metric='rmse', verbose=100, 
        categorical_feature=categories)
    y_predls.append(clf.predict(test_df[cols]))

Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.219511
[200]	training's rmse: 0.213018
[300]	training's rmse: 0.211862
[400]	training's rmse: 0.211508
[500]	training's rmse: 0.211308
[600]	training's rmse: 0.211144
[700]	training's rmse: 0.211005
[800]	training's rmse: 0.210883
[900]	training's rmse: 0.210767
[1000]	training's rmse: 0.210656
[1100]	training's rmse: 0.21055
[1200]	training's rmse: 0.210439
[1300]	training's rmse: 0.210332
[1400]	training's rmse: 0.210226
[1500]	training's rmse: 0.210125
Did not meet early stopping. Best iteration is:
[1502]	training's rmse: 0.210123
Training until validation scores don't improve for 80 rounds.
[100]	training's rmse: 0.219484
[200]	training's rmse: 0.213016
[300]	training's rmse: 0.211859
[400]	training's rmse: 0.211515
[500]	training's rmse: 0.211313
[600]	training's rmse: 0.211153
[700]	training's rmse: 0.211012
[800]	training's rmse: 0.210891
[900]	training's rmse: 0.210773
[1000]	training's rmse

In [34]:
test_df['deal_probability'] = sum(y_predls)/len(y_predls)
test_df['deal_probability'] = np.clip(test_df['deal_probability'], .0001, .9999)
test_df[['item_id', 'deal_probability']].to_csv('../lgbbsub_1606L2.csv.gz', compression='gzip', index=False, header=True)
test_df[['item_id', 'deal_probability']].head()

,item_id,deal_probability
1503424,6544e41a8817,0.437211
1503425,65b9484d670f,0.151174
1503426,8bab230b2ecd,0.124594
1503427,8e348601fefc,0.102618
1503428,8bd2fe400b89,0.167610
